<a href="https://colab.research.google.com/github/annabellesauve/Empathy-Detection/blob/main/dataprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
from transformers import RobertaTokenizer
from google.colab import drive
import codecs
import csv
import numpy as np
import re




In [ ]:
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:

input_file = codecs.open('/content/gdrive/My Drive/empathy_dataset/emotional-reactions-reddit.csv', 'r', 'utf-8')
output_file = codecs.open('/content/gdrive/My Drive/empathy_dataset/edited_emotionalreactions.csv', 'w', 'utf-8')
csv_reader = csv.reader(input_file, delimiter = ',', quotechar='"')
csv_writer = csv.writer(output_file, delimiter = ',',quotechar='"') 

next(csv_reader, None) # skip the headerprint(csv_reader)
csv_writer.writerow(["id","seeker_post","response_post","level","rationale_labels","rationale_labels_trimmed","response_post_masked"])
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)
for row in csv_reader:
	# sp_id,rp_id,seeker_post,response_post,level,rationales

	seeker_post = row[2].strip()
	response = row[3].strip()

	response_masked = response

	response_tokenized = tokenizer.decode(tokenizer.encode_plus(response, add_special_tokens = True, max_length = 64, pad_to_max_length = True)['input_ids'], clean_up_tokenization_spaces=False)

	response_tokenized_non_padded = tokenizer.decode(tokenizer.encode_plus(response, add_special_tokens = True, max_length = 64, pad_to_max_length = False)['input_ids'], clean_up_tokenization_spaces=False)

	response_words = tokenizer.tokenize(response_tokenized)
	response_non_padded_words = tokenizer.tokenize(response_tokenized_non_padded)

	if len(response_words) != 64:
    		  continue

	response_words_position = np.zeros((len(response),), dtype=np.int32)

	rationales = row[5].strip().split('|')

	rationale_labels = np.zeros((len(response_words),), dtype=np.int32)


	curr_position = 0
	for idx in range(len(response_words)):
		curr_word = response_words[idx]
		if curr_word.startswith('Ġ'):
    		    curr_word = curr_word[1:]
		response_words_position[curr_position: curr_position+len(curr_word)+1] = idx
		curr_position += len(curr_word)+1

    
	if len(rationales) == 0 or row[5].strip() == '':
    		rationale_labels[1:len(response_non_padded_words)] = 1
    		response_masked = ''
	for r in rationales:
    		if r == '':
    		    continue
    		try:
    		    r_tokenizer = tokenizer.decode(tokenizer.encode(r, add_special_tokens = False))
    		    match = re.search(r_tokenizer , response_tokenized)

    		    curr_match = response_words_position[match.start(0):match.start(0)+len(r_tokenizer)]
    		    curr_match = list(set(curr_match))
    		    curr_match.sort()

    		    response_masked = response_masked.replace(r, ' ')
    		    response_masked = re.sub(r' +', ' ', response_masked)

    		    rationale_labels[curr_match] = 1
    		except:
    		    continue
    
    
    
	rationale_labels_str = ','.join(str(x) for x in rationale_labels)

	rationale_labels_str_trimmed = ','.join(str(x) for x in rationale_labels[1:len(response_non_padded_words)])


	csv_writer.writerow([row[0] + '_' + row[1], seeker_post, response, row[4], rationale_labels_str, len(rationale_labels_str_trimmed), response_masked])


input_file.close()
output_file.close()
  






Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2217: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
